In [1]:
print('brood is go!')

brood is go!


In [2]:
%matplotlib inline
import time
import pandas as pd
import math
from sklearn.feature_extraction.text import CountVectorizer

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 200

start_time = time.time()

In [3]:
eggs = pd.read_csv('eggs.csv')

usr, twt = [], []
for i in range(len(eggs)):
    usr.append(eggs['User'][i])
    twt.append(eggs['Tweet_Tag'][i])
    
eggs

,User,Tweet_Tag
0,@Nelson_Sayz,Liverpool Sleep Dogs Hike Chess Novels Food Fo...
1,@SanathKumarBS,Cars Sleep Hike Food Tambi TambiMC Food Food F...
2,@RohanThakkar5,Sports Food Food Chess Novels Manchester_Unit...
3,@Rajat_Sethi3,Sex Sex Sex Food Food Women Tambi Agression A...


In [4]:
# Collaborative Filtering 

# Creating dataframe of users against the tags they like
vectorizer = CountVectorizer(min_df=0)

print('Total users considered are : ' + str(len(usr)))
vectorizer.fit(twt)
arr = vectorizer.transform(twt)
arr = arr.toarray()

tag = vectorizer.get_feature_names()
egg = pd.DataFrame(index=range(len(usr)), columns=tag, data=arr, dtype=int)
egg.insert(0, 'user', usr)
egg_orig = egg.copy()

#Dropping the tags which have been associated by only a single user
for i in range(int(len(egg_orig.columns))-1):
    if sum(egg_orig.ix[:,i+1]) == max(egg_orig.ix[:,i+1]):
        egg.drop(egg_orig.columns[i+1],1, inplace=True)

egg

Total users considered are : 4


,user,chess,food,hike,hole,liverpool,novels,sex,sleep,tambi
0,@Nelson_Sayz,1,2,4,0,4,1,0,1,0
1,@SanathKumarBS,0,4,2,4,0,0,1,1,1
2,@RohanThakkar5,1,2,1,2,0,1,0,0,0
3,@Rajat_Sethi3,1,2,1,0,1,0,3,0,1


In [5]:
# Create dataframe and store cosine scores for merchants

tag_mtx = egg.drop('user', 1)
tag_affinity = pd.DataFrame(index=tag_mtx.columns,columns=tag_mtx.columns,dtype=int)

# Fillin in the empty spaces with cosine similarities
# Loop through the columns
from scipy.spatial.distance import cosine
for i in range(0,len(tag_affinity.columns)):
    # Loop through the columns for each column
    for j in range(0,len(tag_affinity.columns)):
        # Fill in placeholder with cosine similarities
        tag_affinity.ix[i,j] = 1-cosine(tag_mtx.ix[:,i],tag_mtx.ix[:,j])
tag_affinity

,chess,food,hike,hole,liverpool,novels,sex,sleep,tambi
chess,1,0.654654,0.738549,0.258199,0.70014,0.816497,0.547723,0.408248,0.408248
food,0.654654,1,0.805823,0.845154,0.458349,0.534522,0.597614,0.801784,0.801784
hike,0.738549,0.805823,1,0.476731,0.879049,0.753778,0.3371,0.904534,0.452267
hole,0.258199,0.845154,0.476731,1,0,0.316228,0.282843,0.632456,0.632456
liverpool,0.70014,0.458349,0.879049,0,1,0.685994,0.230089,0.685994,0.171499
novels,0.816497,0.534522,0.753778,0.316228,0.685994,1,0,0.5,0
sex,0.547723,0.597614,0.3371,0.282843,0.230089,0,1,0.223607,0.894427
sleep,0.408248,0.801784,0.904534,0.632456,0.685994,0.5,0.223607,1,0.5
tambi,0.408248,0.801784,0.452267,0.632456,0.171499,0,0.894427,0.5,1


In [6]:
['%.2f' % elem for elem in list(tag_affinity.ix[0:,i].sort_values(ascending=False)[:5])]

['1.00', '0.89', '0.80', '0.63', '0.50']

In [7]:
# Create placeholder tags for closest neighbours to an tag
tag_neighbours = pd.DataFrame(index=tag_affinity.columns,columns=range(1,6))
tag_neighbourv = pd.DataFrame(index=tag_affinity.columns,columns=range(1,6))

# Loop through our similarity dataframe and fill in neighbouring item tag
for i in range(len(tag_affinity.columns)):
    tag_neighbours.ix[i,:5] = tag_affinity.ix[0:,i].sort_values(ascending=False)[:5].index
    tag_neighbourv.ix[i,:5] = tag_affinity.ix[0:,i].sort_values(ascending=False)[:5
        ].index + " (" + ['%.2f' % elem for elem in list(tag_affinity.ix[0:,i].sort_values(ascending=False)[:5])] + ")"
    
tag_neighbourv

C:\Users\Nelson\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: using '+' to provide set union with Indexes is deprecated, use '|' or .union()


,1,2,3,4,5
chess,chess (1.00),novels (0.82),hike (0.74),liverpool (0.70),food (0.65)
food,food (1.00),hole (0.85),hike (0.81),tambi (0.80),sleep (0.80)
hike,hike (1.00),sleep (0.90),liverpool (0.88),food (0.81),novels (0.75)
hole,hole (1.00),food (0.85),tambi (0.63),sleep (0.63),hike (0.48)
liverpool,liverpool (1.00),hike (0.88),chess (0.70),sleep (0.69),novels (0.69)
novels,novels (1.00),chess (0.82),hike (0.75),liverpool (0.69),food (0.53)
sex,sex (1.00),tambi (0.89),food (0.60),chess (0.55),hike (0.34)
sleep,sleep (1.00),hike (0.90),food (0.80),liverpool (0.69),hole (0.63)
tambi,tambi (1.00),sex (0.89),food (0.80),hole (0.63),sleep (0.50)


In [8]:
# Content Filtering
bro = egg.transpose()
bro.columns = list(bro.ix[0][:])
bro = bro.drop(bro.index[0])

bro

,@Nelson_Sayz,@SanathKumarBS,@RohanThakkar5,@Rajat_Sethi3
chess,1,0,1,1
food,2,4,2,2
hike,4,2,1,1
hole,0,4,2,0
liverpool,4,0,0,1
novels,1,0,1,0
sex,0,1,0,3
sleep,1,1,0,0
tambi,0,1,0,1


In [9]:
# Create dataframe and store cosine scores for merchants
usr_mtx = bro
usr_affinity = pd.DataFrame(index=usr_mtx.columns,columns=usr_mtx.columns,dtype=int)

# Fillin in the empty spaces with cosine similarities
# Loop through the columns
from scipy.spatial.distance import cosine
for i in range(0,len(usr_affinity.columns)):
    # Loop through the columns for each column
    for j in range(0,len(usr_affinity.columns)):
        # Fill in placeholder with cosine similarities
        usr_affinity.ix[i,j] = 1-cosine(usr_mtx.ix[:,i],usr_mtx.ix[:,j])
usr_affinity

,@Nelson_Sayz,@SanathKumarBS,@RohanThakkar5,@Rajat_Sethi3
@Nelson_Sayz,1,0.435897,0.482805,0.504878
@SanathKumarBS,0.435897,1,0.869048,0.543715
@RohanThakkar5,0.482805,0.869048,1,0.438763
@Rajat_Sethi3,0.504878,0.543715,0.438763,1


In [10]:
# Create placeholder item for closest neighbours to other users
usr_neighbours = pd.DataFrame(index=usr_affinity.columns,columns=range(0,3))
usr_neighbourv = pd.DataFrame(index=usr_affinity.columns,columns=range(0,3))

# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(len(usr_affinity.columns)):
    usr_neighbourv.ix[i,:3] = usr_affinity.ix[0:,i].sort_values(ascending=False)[:3
        ].index + " (" + ['%.2f' % elem for elem in list(usr_affinity.ix[0:,i].sort_values(ascending=False)[:3])] + ")"
    
usr_neighbourv.drop(0, 1, inplace=True)
usr_neighbourv.index.name = 'user'
usr_neighbourv.to_csv('bro.csv')
usr_neighbourv

C:\Users\Nelson\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: using '+' to provide set union with Indexes is deprecated, use '|' or .union()


,1,2
user,,
@Nelson_Sayz,@Rajat_Sethi3 (0.50),@RohanThakkar5 (0.48)
@SanathKumarBS,@RohanThakkar5 (0.87),@Rajat_Sethi3 (0.54)
@RohanThakkar5,@SanathKumarBS (0.87),@Nelson_Sayz (0.48)
@Rajat_Sethi3,@SanathKumarBS (0.54),@Nelson_Sayz (0.50)


In [11]:
# Function to calculate collaborative similarity
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities)

In [12]:
# Create a place holder matrix for similarities, and fill in the user name column
tag_lnk = pd.DataFrame(index=egg.index,columns=egg.columns)
tag_lnk.ix[:,:1] = egg.ix[:,:1]
tag_lnk

,user,chess,food,hike,hole,liverpool,novels,sex,sleep,tambi
0,@Nelson_Sayz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,@SanathKumarBS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,@RohanThakkar5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,@Rajat_Sethi3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#Loop through all rows, skip the user column, and fill with similarity scores
for i in range(0,len(tag_lnk.index)):
    for j in range(1,len(tag_lnk.columns)):
        user = tag_lnk.index[i]
        tags = tag_lnk.columns[j]
 
        if egg.ix[i][j] > 0:
            tag_lnk.ix[i][j] = 0
        else:
            tag_top_nam = tag_neighbours.ix[tags][1:5]
            tag_top_sim = tag_affinity.ix[tags].sort_values(ascending=False)[1:5]
            tag_usr_usg = tag_mtx.ix[user,tag_top_nam]
            
            tag_lnk.ix[i][j] = getScore(tag_usr_usg,tag_top_sim)

In [14]:
tag_lnk

,user,chess,food,hike,hole,liverpool,novels,sex,sleep,tambi
0,@Nelson_Sayz,0,0,0,1.63511,0,0,1.3006,0,0.74366
1,@SanathKumarBS,1.40754,0,0,0,0.828175,1.30631,0,0,0
2,@RohanThakkar5,0,0,0,0,0.767552,0,0.875124,1.24737,1.01407
3,@Rajat_Sethi3,0,0,0,1.08222,0,1.19153,0,1.05598,0


In [15]:
# Create Dataframe for top 6 tags
tags_rcmd = pd.DataFrame(index=tag_lnk.index, columns=['user','1','2','3','4','5','6'])
tags_rcmd.ix[0:,0] = tag_lnk.ix[:,0]

# Substituting tag names
for i in range(len(tag_lnk.index)):
    srtd = tag_lnk.ix[i,1:].sort_values(ascending=False).ix[0:6,]
    tg = srtd.index.transpose() + ' ('
    vl = ['%.2f' % elem + ')' for elem in srtd.values.transpose()]
    tags_rcmd.ix[i,1:] = [x + y for x, y in zip(tg, vl)]

tags_rcmd.to_csv('tag.csv', index=False)
tags_rcmd

,user,1,2,3,4,5,6
0,@Nelson_Sayz,hole (1.64),sex (1.30),tambi (0.74),sleep (0.00),novels (0.00),liverpool (0.00)
1,@SanathKumarBS,chess (1.41),novels (1.31),liverpool (0.83),tambi (0.00),sleep (0.00),sex (0.00)
2,@RohanThakkar5,sleep (1.25),tambi (1.01),sex (0.88),liverpool (0.77),novels (0.00),hole (0.00)
3,@Rajat_Sethi3,novels (1.19),hole (1.08),sleep (1.06),tambi (0.00),sex (0.00),liverpool (0.00)


In [16]:
#tag_lnk.ix[i,1:].sort_values(ascending=False).ix[0:6,].values



first=['%.2f' % elem for elem in srtd.values.transpose()]
second=['a','a','a','a','a','a']

[x + y for x, y in zip(first, second)]

['1.19a', '1.08a', '1.06a', '0.00a', '0.00a', '0.00a']

In [17]:
print("Brood Creation: Time taken: " + str(round(time.time() - start_time,2)) + " seconds")

Brood Creation: Time taken: 2.13 seconds
